In [1]:
#!pip install -q kaggle

In [1]:
!mkdir C:\Users\yihim\.kaggle

A subdirectory or file C:\Users\yihim\.kaggle already exists.


In [3]:
!move kaggle.json C:\Users\yihim\.kaggle\

        1 file(s) moved.


In [4]:
!icacls C:\Users\yihim\.kaggle\kaggle.json /inheritance:r

processed file: C:\Users\yihim\.kaggle\kaggle.json
Successfully processed 1 files; Failed processing 0 files


In [5]:
!mkdir C:\Users\yihim\OneDrive\Desktop\ai_mini_projects\object_detection\dataset

In [6]:
!kaggle datasets download -d huanghanchina/pascal-voc-2012 -p C:\Users\yihim\OneDrive\Desktop\ai_mini_projects\object_detection\dataset


  0%|          | 0.00/3.63G [00:00<?, ?B/s]
  0%|          | 1.00M/3.63G [00:00<57:56, 1.12MB/s]
  0%|          | 2.00M/3.63G [00:01<29:40, 2.19MB/s]
  0%|          | 4.00M/3.63G [00:01<14:11, 4.57MB/s]
  0%|          | 7.00M/3.63G [00:01<07:36, 8.53MB/s]
  0%|          | 9.00M/3.63G [00:01<06:00, 10.8MB/s]
  0%|          | 12.0M/3.63G [00:01<04:32, 14.2MB/s]
  0%|          | 15.0M/3.63G [00:01<03:51, 16.8MB/s]
  0%|          | 18.0M/3.63G [00:01<03:25, 18.9MB/s]
  1%|          | 21.0M/3.63G [00:02<03:13, 20.0MB/s]
  1%|          | 24.0M/3.63G [00:02<03:06, 20.8MB/s]
  1%|          | 27.0M/3.63G [00:02<02:58, 21.7MB/s]
  1%|          | 30.0M/3.63G [00:02<03:17, 19.6MB/s]
  1%|          | 33.0M/3.63G [00:02<03:11, 20.1MB/s]
  1%|          | 37.0M/3.63G [00:02<02:59, 21.5MB/s]
  1%|1         | 41.0M/3.63G [00:02<02:50, 22.6MB/s]
  1%|1         | 45.0M/3.63G [00:03<02:45, 23.3MB/s]
  1%|1         | 48.0M/3.63G [00:03<02:49, 22.7MB/s]
  1%|1         | 52.0M/3.63G [00:03<02:58, 21.5MB/s]
 

In [7]:
import zipfile
zip_ref = zipfile.ZipFile("dataset/pascal-voc-2012.zip")
zip_ref.extractall("dataset/")
zip_ref.close()

In [1]:
import tensorflow as tf### models
import numpy as np### math computations
import seaborn as sns### visualizations
import matplotlib.pyplot as plt### plotting bar chart
import datetime
import pathlib
import io
from datetime import datetime
import json
import xml.etree.ElementTree as ET
import os
import shutil
import cv2
import time
import random
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

In [2]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

In [76]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [78]:
!mkdir dataset\VOC2012\ValJPEGImages\
!mkdir dataset\VOC2012\ValAnnotations\

In [122]:
train_images = "dataset/VOC2012/JPEGImages/"
train_maps = "dataset/VOC2012/Annotations/"

val_images = "dataset/VOC2012/ValJPEGImages/"
val_maps = "dataset/VOC2012/ValAnnotations/"

# for i in val_list:
#     shutil.move(train_images+i, val_images+i)

# for i in val_list:
#     shutil.move(train_maps+i[:-3]+"xml", val_maps+i[:-3]+"xml")

classes = ['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B = 2
N_CLASSES = len(classes)
H, W = 224, 224
SPLIT_SIZE = int(H/32)
N_EPOCHS = 135
LR = 5e-4
BATCH_SIZE = 32

In [70]:
def preprocess_xml(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    size_tree = root.find('size')
    height = float(size_tree.find('height').text)
    # print(height)
    width = float(size_tree.find('width').text)
    # print(width)
    # depth = float(size_tree.find('depth').text)
    # print(depth)
    bounding_boxes = []
    for object_tree in root.findall('object'):
        for bounding_box in object_tree.iter('bndbox'):
            xmin = float(bounding_box.find('xmin').text)
            ymin = float(bounding_box.find('ymin').text)
            xmax = float(bounding_box.find('xmax').text)
            ymax = float(bounding_box.find('ymax').text)
            # the coordinates of every image at the most top left is (0, 0)
            # xmin & ymin are the coordinate of the image at the top left
            # xmax & ymax are the coordinates of the image ar the bottom right
            # print(xmin,ymin,xmax,ymax)
            break
        class_name = object_tree.find('name').text
        # Label encoding the class name from (str) to (int)
        class_dict = {classes[i]:i for i in range(len(classes))}
        # print(class_dict)
        # Performing normalization based on the different sizes of image
        # the (xmin+xmax)/(2*width) is to get the center coordinates of the bounding box then normalize it
        bounding_box = [
            (xmin+xmax)/(2*width), (ymin+ymax)/(2*height), (xmax-xmin)/width,
            (ymax-ymin)/height, class_dict[class_name]
        ]
        bounding_boxes.append(bounding_box)
        # print(bounding_boxes)

    return tf.convert_to_tensor(bounding_boxes)

In [71]:
preprocess_xml(train_maps+"2007_000032.xml")

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.479     ,  0.4644128 ,  0.542     ,  0.37366548,  0.        ],
       [ 0.33      ,  0.37544483,  0.128     ,  0.12455516,  0.        ],
       [ 0.408     ,  0.727758  ,  0.036     ,  0.17437722, 14.        ],
       [ 0.07      ,  0.7597865 ,  0.036     ,  0.17437722, 14.        ]],
      dtype=float32)>

In [74]:
def generate_output(bounding_boxes):
    output_label=np.zeros((int(SPLIT_SIZE),int(SPLIT_SIZE),N_CLASSES+5))

    for b in range(len(bounding_boxes)):
        # this is to get the position of the grid cell that contains the object's center position
        grid_x = bounding_boxes[...,b,0]*SPLIT_SIZE # multiplying 7 to get the position from the normalized position
        grid_y = bounding_boxes[...,b,1]*SPLIT_SIZE
        # converting to int to locate the position of the grid cell
        # eg. i = from 0.49 to 0, j = from 5.20 to 5 (in this case the image is separated to 7 * 7 grid)
        i = int(grid_x)
        j = int(grid_y)

        # this is to label the first 5 values
        output_label[i,j,0:5] = [1., grid_x%1, grid_y%1, bounding_boxes[...,b,2], bounding_boxes[...,b,3]]
        # this is to label the class from the 20 classes, eg. 00000000000001000000 af the the first 5 values
        output_label[i,j,5+int(bounding_boxes[...,b,4])] = 1.
        # in the end the output label will be like, eg. 1,0.49,0.2,0.036,0.17,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0

    return tf.convert_to_tensor(output_label, dtype=tf.float32)

In [81]:
generate_output(preprocess_xml(val_maps+"2007_000032.xml"),)[0][5]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1.        , 0.49      , 0.3185053 , 0.036     , 0.17437722,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)>

In [191]:
img_paths = []
xml_paths = []

val_img_paths = []
val_xml_paths = []

for i in os.listdir(train_maps):
    img_paths.append(train_images+i[:-3]+"jpg")
    xml_paths.append(train_maps+i)

for i in os.listdir(val_maps):
    val_img_paths.append(val_images+i[:-3]+"jpg")
    val_xml_paths.append(val_maps+i)

print(len(img_paths), len(xml_paths))
print(len(val_img_paths), len(val_xml_paths))

17061 17061
64 64


In [192]:
train_dataset = tf.data.Dataset.from_tensor_slices((img_paths, xml_paths))
val_dataset = tf.data.Dataset.from_tensor_slices((val_img_paths, val_xml_paths))

In [193]:
for i in val_dataset.take(1):
    print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'dataset/VOC2012/ValJPEGImages/2007_000027.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'dataset/VOC2012/ValAnnotations/2007_000027.xml'>)


In [194]:
def get_img_bboxes(img_path, xml_path):
    img = tf.io.decode_jpeg(tf.io.read_file(img_path))
    img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

    # Specifying the input to use the box_path and output dtype to float32
    bboxes = tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)

    return img, bboxes

In [195]:
train_dataset = train_dataset.map(map_func=get_img_bboxes, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(map_func=get_img_bboxes, num_parallel_calls=tf.data.AUTOTUNE)

In [196]:
# Visualize the data

In [197]:
for i, j in train_dataset.skip(18):
    print(i.shape, j)
    break

(224, 224, 3) tf.Tensor(
[[0.494      0.41666666 0.716      0.3888889  0.        ]
 [0.854      0.8611111  0.28       0.13157895 0.        ]
 [0.512      0.85672516 0.24       0.12280702 0.        ]], shape=(3, 5), dtype=float32)


In [198]:
cv2.imwrite('out_1.jpg', np.array(i))

True

In [199]:
def preprocess_bboxes(img, bboxes):
    labels = tf.numpy_function(func=generate_output, inp=[bboxes], Tout=tf.float32)
    return img, labels

In [200]:
train_dataset = train_dataset.map(preprocess_bboxes, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_bboxes, num_parallel_calls=tf.data.AUTOTUNE)

In [201]:
for i,j in train_dataset.take(1):
    print(i.shape, j)

(224, 224, 3) tf.Tensor(
[[[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  ...
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.         0.         0.         ... 0.         0.         0.       

In [202]:
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [146]:
NUM_FILTERS = 512
# the B = 2, is used to predict whether the image is overlapped with multiple objects
# then based on the confidence, to determine which class
OUTPUT_DIM = N_CLASSES + 5 * B

In [147]:
OUTPUT_DIM

30

In [374]:
# base_model = tf.keras.applications.resnet50.ResNet50(weights="imagenet", 
#                                                      input_shape=(H, W, 3), 
#                                                      include_top=False)
base_model = tf.keras.applications.efficientnet.EfficientNetB1(weights="imagenet",
                                                               input_shape=(H, W, 3),
                                                               include_top=False)

base_model.trainable=False

27033600/27018416 [==============================] - 3s 0us/step


In [375]:
base_model.summary()

Model: "efficientnetb1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0

In [409]:
tf.keras.backend.clear_session()

model = tf.keras.Sequential([
    base_model,
    Conv2D(NUM_FILTERS, 3, padding="same", kernel_initializer="he_normal"),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Conv2D(NUM_FILTERS, 3, padding="same", kernel_initializer="he_normal"),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Conv2D(NUM_FILTERS, 3, padding="same", kernel_initializer="he_normal"),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Conv2D(NUM_FILTERS, 3, padding="same", kernel_initializer="he_normal"),
    LeakyReLU(alpha=0.1),
    Flatten(),
    Dense(NUM_FILTERS, kernel_initializer="he_normal"),
    BatchNormalization(),
    LeakyReLU(alpha=0.1),
    Dropout(0.5),
    Dense(int(SPLIT_SIZE)*int(SPLIT_SIZE)*OUTPUT_DIM, activation="sigmoid"),
    Reshape((int(SPLIT_SIZE), int(SPLIT_SIZE), OUTPUT_DIM))
], name="yolo_object_detection_pascal_voc_2012")

model.summary()

Model: "yolo_object_detection_pascal_voc_2012"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 7, 7, 1280)       6575239   
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 512)         5898752   
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 7, 512)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 512)         2359808   
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 512)        2048      
 hNormalization)             

In [377]:
CHECKPOINT_PATH = "yolo_efficientnetB1.h5"
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                   save_weights_only=True,
                                                   save_best_only=True,
                                                   monitor="val_loss",
                                                   # to choose the minimum of the val_loss
                                                   mode="min")

In [413]:
def scheduler(epoch, lr):
    if epoch < 40:
        return 1e-3
    elif epoch >=40 and epoch<80:
        return 5e-4
    else:
        return 1e-4

In [379]:
lr_scheduler_cb = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [380]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    intersection = tf.maximum(0.0, rd - lu)
    inter_square = intersection[..., 0] * intersection[..., 1]

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    union_square = tf.maximum(square1 + square2 - inter_square, 1e-10)
    return tf.clip_by_value(inter_square / union_square, 0.0, 1.0)

In [381]:
def difference(x,y):
  return tf.reduce_sum(tf.square(y-x))

In [382]:
def yolo_loss(y_true, y_pred):
  target = y_true[...,0]

  ###################### OBject Loss
  y_pred_extract = tf.gather_nd(y_pred, tf.where(target[:]==1))
  y_target_extract = tf.gather_nd(y_true, tf.where(target[:]==1))
  
  rescaler = tf.where(target[:]==1)*SPLIT_SIZE
  upscaler_1 = tf.concat([rescaler[:,1:],tf.zeros([len(rescaler),2], dtype=tf.int64)],axis=-1)
  
  target_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                       repeats=[len(rescaler)], axis=0)*tf.cast(y_target_extract[...,1:5], dtype = tf.float32)
  pred_1_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,1:5], dtype = tf.float32)
  pred_2_upscaler_2 = tf.repeat([[float(SPLIT_SIZE),float(SPLIT_SIZE),H,W]],
                      repeats=[len(rescaler)], axis=0)*tf.cast(y_pred_extract[...,6:10], dtype = tf.float32)
  
  target_orig = tf.cast(upscaler_1, dtype = tf.float32)+target_upscaler_2
  pred_1_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_1_upscaler_2
  pred_2_orig = tf.cast(upscaler_1, dtype = tf.float32)+pred_2_upscaler_2
  
  mask =tf.cast(tf.math.greater(compute_iou(target_orig,pred_2_orig),
                                         compute_iou(target_orig,pred_1_orig)),dtype=tf.int32)
  
  y_pred_joined=tf.transpose(tf.concat([tf.expand_dims(y_pred_extract[...,0],axis=0),
                        tf.expand_dims(y_pred_extract[...,5],axis=0)],axis=0))
  
  obj_pred = tf.gather_nd(y_pred_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  
  object_loss = difference(tf.cast(obj_pred,dtype =tf.float32)
                            ,tf.cast(tf.ones([len(rescaler)]),dtype=tf.float32))

  ####################### For No object
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==0))
  y_target_extract = tf.zeros(len(y_pred_extract))

  no_object_loss_1 = difference(tf.cast(y_pred_extract[...,0],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))
  
  no_object_loss_2 = difference(tf.cast(y_pred_extract[...,5],dtype =tf.float32)
                            ,tf.cast(y_target_extract,dtype=tf.float32))
  
  no_object_loss = no_object_loss_1+no_object_loss_2

  ######################## For OBject class loss
  y_pred_extract = tf.gather_nd(y_pred[...,10:],tf.where(target[:]==1))
  class_extract = tf.gather_nd(y_true[...,5:],tf.where(target[:]==1))

  class_loss = difference(tf.cast(y_pred_extract,dtype =tf.float32)
                                ,tf.cast(class_extract,dtype=tf.float32))

  ######################### For object bounding box loss
  y_pred_extract = tf.gather_nd(y_pred[...,0:B*5], tf.where(target[:]==1))
  centre_joined=tf.stack([y_pred_extract[...,1:3],y_pred_extract[...,6:8]],axis=1)
  centre_pred = tf.gather_nd(centre_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  centre_target = tf.gather_nd(y_true[...,1:3], tf.where(target[:]==1))
  
  centre_loss = difference(centre_pred,centre_target)
    
  size_joined=tf.stack([y_pred_extract[...,3:5],y_pred_extract[...,8:10]],axis=1)

  size_pred = tf.gather_nd(size_joined,tf.stack([tf.range(len(rescaler)),mask],axis=-1))
  size_target = tf.gather_nd(y_true[...,3:5], tf.where(target[:]==1))
  
  size_loss = difference(tf.math.sqrt(tf.math.abs(size_pred)),tf.math.sqrt(tf.math.abs(size_target)))
  box_loss = centre_loss+size_loss
  
  lambda_coord = 5.0
  lambda_no_obj = 0.5

  loss = object_loss + (lambda_no_obj*no_object_loss)+ tf.cast(lambda_coord*box_loss,dtype=tf.float32)+ tf.cast(class_loss,dtype=tf.float32) 
  return loss

In [161]:
model.compile(loss=yolo_loss,
              optimizer=Adam(1e-3))

In [163]:
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=N_EPOCHS,
                    verbose=1,
                    callbacks=[lr_scheduler_cb, checkpoint_cb])

Epoch 1/135
534/534 [==============================] - 144s 261ms/step - loss: 205.0403 - val_loss: 246.6320 - lr: 0.0010
Epoch 2/135
534/534 [==============================] - 136s 255ms/step - loss: 181.6708 - val_loss: 240.1902 - lr: 0.0010
Epoch 3/135
534/534 [==============================] - 145s 271ms/step - loss: 177.2217 - val_loss: 240.7016 - lr: 0.0010
Epoch 4/135
534/534 [==============================] - 140s 262ms/step - loss: 174.4182 - val_loss: 231.2072 - lr: 0.0010
Epoch 5/135
534/534 [==============================] - 143s 268ms/step - loss: 172.0852 - val_loss: 226.6986 - lr: 0.0010
Epoch 6/135
534/534 [==============================] - 168s 315ms/step - loss: 170.1196 - val_loss: 233.4116 - lr: 0.0010
Epoch 7/135
 49/534 [=>............................] - ETA: 2:10 - loss: 203.5012

KeyboardInterrupt: 

In [164]:
# Overfitting, need to perform data augmentation

In [342]:
transform = A.Compose([
    A.Resize(H, W),
    A.RandomCrop(width=np.random.randint(int(0.9*W), W),
                 height=np.random.randint(int(0.9*H), H), 
                 p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4, p=0.5),
    A.HorizontalFlip(p=0.5),
    A.Resize(H, W)
], bbox_params=A.BboxParams(format="yolo")) # min_area=25, min_visibility=0.1 - this is to specify whether to detect the object based on the min value

In [343]:
def aug_albument(image, bboxes):
    augmented = transform(image=image, bboxes=bboxes)
    return [
        tf.convert_to_tensor(augmented["image"], dtype=tf.float32),
        tf.convert_to_tensor(augmented["bboxes"], dtype=tf.float32)
    ]

In [344]:
def process_aug(image, bboxes):
    aug = tf.numpy_function(func=aug_albument, inp=[image, bboxes], Tout=[tf.float32, tf.float32])
    return aug[0], aug[1]

In [386]:
train_dataset_aug = tf.data.Dataset.from_tensor_slices((img_paths, xml_paths))

In [387]:
train_dataset_aug = train_dataset_aug.map(get_img_bboxes, num_parallel_calls=tf.data.AUTOTUNE)

In [388]:
train_dataset_aug

<ParallelMapDataset element_spec=(TensorSpec(shape=(224, 224, None), dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [389]:
# Before aug

In [390]:
for i, j in train_dataset_aug.skip(18):
    print(i.shape, j)
    break

(224, 224, 3) tf.Tensor(
[[0.494      0.41666666 0.716      0.3888889  0.        ]
 [0.854      0.8611111  0.28       0.13157895 0.        ]
 [0.512      0.85672516 0.24       0.12280702 0.        ]], shape=(3, 5), dtype=float32)


In [391]:
train_dataset_aug = train_dataset_aug.map(process_aug, num_parallel_calls=tf.data.AUTOTUNE)

In [392]:
train_dataset_aug

<ParallelMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), TensorSpec(shape=<unknown>, dtype=tf.float32, name=None))>

In [393]:
# After aug

In [394]:
for i, j in train_dataset_aug.skip(18):
    print(i.shape, j)
    break

(224, 224, 3) tf.Tensor(
[[0.50600004 0.41666666 0.716      0.3888889  0.        ]
 [0.14600003 0.8611111  0.28       0.13157895 0.        ]
 [0.48799998 0.85672516 0.24       0.12280702 0.        ]], shape=(3, 5), dtype=float32)


In [395]:
cv2.imwrite("out_2.jpg", np.array(i))

True

In [396]:
def preprocess_aug_img_bboxes(image, bboxes):
    image = tf.image.random_brightness(image, max_delta=50.)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)
    # image = tf.image.random_hue(image, max_delta=0.5)
    image = tf.clip_by_value(image, 0, 255)
    labels = tf.numpy_function(func=generate_output, inp=[bboxes], Tout=tf.float32)
    return image, labels

In [397]:
train_dataset_aug = train_dataset_aug.map(preprocess_aug_img_bboxes, num_parallel_calls=tf.data.AUTOTUNE)

In [398]:
val_dataset = tf.data.Dataset.from_tensor_slices((val_img_paths, val_xml_paths))
val_dataset = val_dataset.map(map_func=get_img_bboxes, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(preprocess_bboxes, num_parallel_calls=tf.data.AUTOTUNE)

In [400]:
for i, j in train_dataset_aug:
    assert i.shape == (224, 224, 3)
    assert j.shape == (7, 7, 25)

In [401]:
for i, j in val_dataset:
    assert i.shape == (224, 224, 3)
    assert j.shape == (7, 7, 25)

In [402]:
train_dataset_aug = train_dataset_aug.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [403]:
for i, j in train_dataset_aug.take(1):
    print(i.shape, j)

(32, 224, 224, 3) tf.Tensor(
[[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.  

In [404]:
cv2.imwrite("out_3.jpg", np.array(i[18]))

True

In [410]:
model.summary()

Model: "yolo_object_detection_pascal_voc_2012"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb1 (Functional)  (None, 7, 7, 1280)       6575239   
                                                                 
 conv2d (Conv2D)             (None, 7, 7, 512)         5898752   
                                                                 
 batch_normalization (BatchN  (None, 7, 7, 512)        2048      
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 7, 7, 512)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 512)         2359808   
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 512)        2048      
 hNormalization)             

In [411]:
model.compile(loss=yolo_loss,
              optimizer=Adam(1e-3))

In [412]:
history = model.fit(train_dataset_aug,
                    validation_data=val_dataset,
                    epochs=135,
                    verbose=1,
                    callbacks=[lr_scheduler_cb, checkpoint_cb])

Epoch 1/135
534/534 [==============================] - 255s 446ms/step - loss: 190.6395 - val_loss: 242.4024 - lr: 0.0010
Epoch 2/135
534/534 [==============================] - 213s 398ms/step - loss: 157.7768 - val_loss: 216.5890 - lr: 0.0010
Epoch 3/135
534/534 [==============================] - 228s 427ms/step - loss: 148.8553 - val_loss: 198.4664 - lr: 0.0010
Epoch 4/135
534/534 [==============================] - 197s 370ms/step - loss: 141.4853 - val_loss: 200.7722 - lr: 0.0010
Epoch 5/135
534/534 [==============================] - 196s 367ms/step - loss: 136.2104 - val_loss: 190.1952 - lr: 0.0010
Epoch 6/135
534/534 [==============================] - 196s 367ms/step - loss: 131.2678 - val_loss: 179.6812 - lr: 0.0010
Epoch 7/135
534/534 [==============================] - 214s 401ms/step - loss: 127.2133 - val_loss: 182.6741 - lr: 0.0010
Epoch 8/135
534/534 [==============================] - 217s 406ms/step - loss: 123.8350 - val_loss: 175.1276 - lr: 0.0010
Epoch 9/135
534/534 [===

KeyboardInterrupt: 

In [414]:
model.load_weights(CHECKPOINT_PATH)

In [415]:
model.evaluate(train_dataset_aug), model.evaluate(val_dataset)

2/2 [==============================] - 1s 235ms/step - loss: 143.2058


(90.52981567382812, 143.205810546875)

In [418]:
model.save("pascal_voc_2012_yolo_efficientnetB1.h5")